In [1]:
import pandas as pd
import numpy as np
import requests
import json
!pip install fsspec
print('Setup Complete.')

Setup Complete.


In [3]:
response = requests.get("https://api.collegefootballdata.com/teams")
teams = pd.read_json(response.text)

teams.head()

ValueError: Protocol not known: [{"id":2000,"school":"Abilene Christian","mascot":"Wildcats","abbreviation":"ACU","alt_name1":null,"alt_name2":"ACU","alt_name3":"Abil Christian","conference":null,"division":null,"color":"#4e2683","alt_color":"#ebebeb","logos":["http

In [4]:
response = requests.get(
    "https://api.collegefootballdata.com/games",
    params = {"year":2019, "seasonType":"both"}
)

data = pd.read_json(response.text)
data.head()

,id,season,week,season_type,start_date,start_time_tbd,neutral_site,conference_game,attendance,venue_id,...,home_points,home_line_scores,home_post_win_prob,away_id,away_team,away_conference,away_points,away_line_scores,away_post_win_prob,excitement_index
0,401110723,2019,1,regular,2019-08-24T23:00:00.000Z,NaN,True,False,66543.0,4013,...,24,"[7, 0, 10, 7]",0.905953,2390,Miami,ACC,20,"[3, 10, 0, 7]",0.094047,8.767910
1,401114164,2019,1,regular,2019-08-25T02:30:00.000Z,NaN,False,False,22396.0,3610,...,45,"[14, 14, 7, 10]",0.688630,12,Arizona,Pac-12,38,"[0, 21, 14, 3]",0.311370,7.842417
2,401119254,2019,1,regular,2019-08-29T23:00:00.000Z,NaN,False,False,17620.0,3700,...,46,"[13, 17, 7, 9]",0.999979,2415,Morgan State,None,3,"[0, 3, 0, 0]",0.000021,0.472968
3,401117855,2019,1,regular,2019-08-29T23:00:00.000Z,NaN,False,False,19648.0,3892,...,24,"[7, 3, 14, 0]",0.728942,2681,Wagner,None,21,"[0, 0, 14, 7]",0.271058,1.834351
4,401119255,2019,1,regular,2019-08-29T23:00:00.000Z,NaN,False,False,18412.0,3965,...,38,"[21, 7, 10, 0]",0.999788,2523,Robert Morris,None,10,"[7, 3, 0, 0]",0.000212,0.118588


In [8]:
pd.set_option("display.max_columns", 500)

In [9]:
data.head()

,id,season,week,season_type,start_date,start_time_tbd,neutral_site,conference_game,attendance,venue_id,venue,home_id,home_team,home_conference,home_points,home_line_scores,home_post_win_prob,away_id,away_team,away_conference,away_points,away_line_scores,away_post_win_prob,excitement_index
0,401110723,2019,1,regular,2019-08-24T23:00:00.000Z,NaN,True,False,66543.0,4013,Camping World Stadium,57,Florida,SEC,24,"[7, 0, 10, 7]",0.905953,2390,Miami,ACC,20,"[3, 10, 0, 7]",0.094047,8.767910
1,401114164,2019,1,regular,2019-08-25T02:30:00.000Z,NaN,False,False,22396.0,3610,Aloha Stadium,62,Hawai'i,Mountain West,45,"[14, 14, 7, 10]",0.688630,12,Arizona,Pac-12,38,"[0, 21, 14, 3]",0.311370,7.842417
2,401119254,2019,1,regular,2019-08-29T23:00:00.000Z,NaN,False,False,17620.0,3700,Doyt Perry Stadium,189,Bowling Green,Mid-American,46,"[13, 17, 7, 9]",0.999979,2415,Morgan State,None,3,"[0, 3, 0, 0]",0.000021,0.472968
3,401117855,2019,1,regular,2019-08-29T23:00:00.000Z,NaN,False,False,19648.0,3892,Rentschler Field,41,Connecticut,American Athletic,24,"[7, 3, 14, 0]",0.728942,2681,Wagner,None,21,"[0, 0, 14, 7]",0.271058,1.834351
4,401119255,2019,1,regular,2019-08-29T23:00:00.000Z,NaN,False,False,18412.0,3965,UB Stadium,2084,Buffalo,Mid-American,38,"[21, 7, 10, 0]",0.999788,2523,Robert Morris,None,10,"[7, 3, 0, 0]",0.000212,0.118588


In [10]:
data = data[
    (pd.notna(data['home_conference'])) & (pd.notna(data['away_conference']))
]

In [11]:
data.shape

(774, 24)

In [12]:
data.isnull().sum().sort_values(ascending=False)

start_time_tbd        774
excitement_index        1
attendance              1
away_post_win_prob      0
season                  0
week                    0
season_type             0
start_date              0
neutral_site            0
conference_game         0
venue_id                0
venue                   0
home_id                 0
home_team               0
home_conference         0
home_points             0
home_line_scores        0
home_post_win_prob      0
away_id                 0
away_team               0
away_conference         0
away_points             0
away_line_scores        0
id                      0
dtype: int64

In [13]:
data = data[
    (data['home_points'] == data['home_points']) & (data['away_points'] == data['away_points'])
    & (pd.notna(data['home_conference'])) & (pd.notna(data['away_conference']))
]

In [14]:
data['home_spread'] = np.where(data['neutral_site'] == True, data['home_points'] - data['away_points'], (data['home_points'] - data['away_points'] - 2.5))
data['away_spread'] = -data['home_spread']
data.head()

,id,season,week,season_type,start_date,start_time_tbd,neutral_site,conference_game,attendance,venue_id,venue,home_id,home_team,home_conference,home_points,home_line_scores,home_post_win_prob,away_id,away_team,away_conference,away_points,away_line_scores,away_post_win_prob,excitement_index,home_spread,away_spread
0,401110723,2019,1,regular,2019-08-24T23:00:00.000Z,NaN,True,False,66543.0,4013,Camping World Stadium,57,Florida,SEC,24,"[7, 0, 10, 7]",0.905953,2390,Miami,ACC,20,"[3, 10, 0, 7]",0.094047,8.767910,4.0,-4.0
1,401114164,2019,1,regular,2019-08-25T02:30:00.000Z,NaN,False,False,22396.0,3610,Aloha Stadium,62,Hawai'i,Mountain West,45,"[14, 14, 7, 10]",0.688630,12,Arizona,Pac-12,38,"[0, 21, 14, 3]",0.311370,7.842417,4.5,-4.5
5,401117854,2019,1,regular,2019-08-29T23:00:00.000Z,NaN,False,False,38032.0,3854,Nippert Stadium,2132,Cincinnati,American Athletic,24,"[7, 3, 7, 7]",0.996829,26,UCLA,Pac-12,14,"[0, 7, 7, 0]",0.003171,5.498682,7.5,-7.5
9,401114236,2019,1,regular,2019-08-30T00:00:00.000Z,NaN,False,False,16361.0,4729,Benson Field at Yulman Stadium,2655,Tulane,American Athletic,42,"[7, 21, 14, 0]",0.999668,2229,Florida International,Conference USA,14,"[0, 7, 7, 0]",0.000332,1.528502,25.5,-25.5
11,401111653,2019,1,regular,2019-08-30T00:00:00.000Z,NaN,False,True,79118.0,3836,Memorial Stadium,228,Clemson,ACC,52,"[14, 14, 14, 10]",0.999976,59,Georgia Tech,ACC,14,"[0, 0, 7, 7]",0.000024,0.121090,35.5,-35.5
